# Распознование рукописных цифр (MNIST) с помощью RBM

### 1. Подключение библиотек

In [1]:
import os
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import numpy as np
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

### 2. Загрузка набора данных MNIST

__Скачивание данных в текущую директорию:__

In [2]:
dir_name = os.getcwd()

__Чтение тренировочной и тестовой выборок набора данных MNIST :__

Данные представляются в виде пар __(tuple)__, где первый элемент — изображение в формате __PIL.Image.Image__, а второй — целочисленная метка класса. Параметр __transform__ обеспечивает преобразование изображений в формат __torch.Tensor__ для последующей работы.

In [3]:
train_dataset = torchvision.datasets.MNIST(root = dir_name, train = True, 
download = True, transform = torchvision.transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root = dir_name, train = False, 
download = True, transform = torchvision.transforms.ToTensor())

__Зададим размер обрабатываемой пачки данных:__

In [4]:
batch_size = 64

__Создадим объекты для последовательной загрузки пачек данных из тренировочной и тестовой выборок:__

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

### 3. Создание модели, соответствующей ограниченной машине Больцмана

__Зададим количество нейронов видимого слоя:__ 28 * 28 = 784, поскольку изображения имеют размер 28 на 28 пикселей

In [6]:
visible_units = 28 * 28

__Зададим количество нейронов скрытого слоя:__

In [7]:
hidden_units = 128

__Зададим скорость обучения модели:__

In [8]:
learning_rate = 0.001

__Зададим количество эпох обучения модели:__

In [9]:
num_epochs = 10

__Зададим параметр алгоритма контрастной дивергенции:__

In [10]:
CD_k = 2

__Зададим коэффициент затухания весов:__

In [11]:
weight_decay = 0.0001

__Зададим коэффициент метода импульса:__

In [12]:
momentum_coefficient = 0.5

__Подробнее о MNIST__

* __train_dataset__ содержит 60.000, а __test_dataset__ — 10.000 изображений в оттенках серого.
* Каждый пиксель изображения принимает действительное значение в отрезке от 0 до 1.
* При __batch_size = 64__ обучение модели займет 937 итераций (64 * 937 = 59968) с размером пачки 64 и одну итерацию с размером пачки 32.

__Создадим класс сети, соответствующей ограниченной машине Больцмана:__

In [13]:
class RBM():
    # Конструктор
    def __init__(self, num_visible, num_hidden, k, learning_rate, momentum_coefficient, weight_decay):
        # Задаем количество нейронов видимого и скрытого слоев, параметр алгоритма
        # контрастной дивергенции, скорость обучения модели, коэффициенты
        # затухания весов и метода импульса
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.k = k
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.momentum_coefficient = momentum_coefficient

        # Задаем вектора смещений и матрицу связей (заполняем их случайными
        # числами из стандартного нормального распределения)
        self.weights = torch.randn(num_visible, num_hidden)
        self.visible_bias = torch.randn(num_visible)
        self.hidden_bias = torch.randn(num_hidden)
        
        # Задаем вектора смещений и матрицу связей для метода моментов
        # (заполняем их нулями)
        self.weights_momentum = torch.zeros(num_visible, num_hidden)
        self.visible_bias_momentum = torch.zeros(num_visible)
        self.hidden_bias_momentum = torch.zeros(num_hidden)

    # Будем считать, что W - матрица связей, a и b - вектора смещений 
    # для видимых и скрытых нейронов соответственно, v - вектор состояния
    # нейронов видимого слоя, h - вектор состояний нейронов скрытого слоя
    
    # Тогда, в соответствии с введенными обозначениями:
    
    # Функция принимает на вход вектор v и вычисляет вектор p(h|v) = sigmoid(v * W + b)
    def sample_hidden(self, visible_probabilities):
        # visible_probabilities ∈ R^(64 × 784) на всех итерациях, кроме последней (на ней 32 × 784)
        # W ∈ R^(784 × 128), b ∈ R^(1 × 128), visible_probabilities * W ∈ R^(64 × 128)
        # visible_probabilities * W + b ∈ R^(64 × 128), т.е вектор смещений b прибавляется
        # к каждой строчке (каждому изображению, поскольку их на данной итерации как раз 64)
        hidden_activations = torch.matmul(visible_probabilities, self.weights) + self.hidden_bias
        
        # sigmoid вычисляется от каждого элемента полученной на предыдущем шаге матрицы
        hidden_probabilities = self._sigmoid(hidden_activations)
        
        return hidden_probabilities # ∈ R^(64 × 128)
    
    # Функция принимает на вход вектор h и вычисляет вектор p(v|h) = sigmoid(h * W^T + a)
    def sample_visible(self, hidden_probabilities):
        # hidden_probabilities ∈ R^(64 × 128) на всех итерациях, кроме последней (на ней 32 × 128)
        # W^T ∈ R^(128 × 784), a ∈ R^(1 × 784), hidden_probabilities * W^T ∈ R^(64 × 784)
        # hidden_probabilities * W^T + a ∈ R^(64 × 784), т.е вектор смещений a прибавляется
        # к каждой строчке (каждому изображению, поскольку их на данной итерации как раз 64)
        visible_activations = torch.matmul(hidden_probabilities, self.weights.t()) + self.visible_bias
        
        # sigmoid вычисляется от каждого элемента полученной на предыдущем шаге матрицы
        visible_probabilities = self._sigmoid(visible_activations)
        
        return visible_probabilities # ∈ R^(64 × 784)
    
    # Стоит отметить тот факт, что данные формулы для вычисления условных вероятностей
    # отличаются от стандартных: p(h|v) = sigmoid(W^T * v + b) и p(v|h) = sigmoid(W * h + a)
    # Это происходит из-за того, что в данной реализации считается, что вектора - это строки,
    # а не столбцы, и если данные выражения для условных вероятностей транспонировать, то
    # можно убедиться, что формулы согласуются с теорией
    
    def contrastive_divergence(self, input_data):
        # Вычисляем вектор p(h|v)
        # input_data ∈ R^(64 × 784)
        positive_hidden_probabilities = self.sample_hidden(input_data) # ∈ R^(64 × 128)
        
        # Активируем нейроны следующим способом:
        # Генерируется вектор с той же длины с случайными числами
        # из стандартного нормального распределения, после чего
        # вектор p(h|v), полученный ранее, покомпонентно с ним сравнивается,
        # в результате чего получаем вектор, состоящий из 0 и 1, где 1 соответствует
        # нейрону, который активировался
        
        # positive_hidden_probabilities ∈ R^(64 × 128), _random_probabilities ∈ R^(1 × 128), из-за чего он будет вызван 64 раза
        # полученный вектор будет состоять из 1.0 и 0.0 (без .float() было бы True и False)
        positive_hidden_activations = (positive_hidden_probabilities 
                                       >= self._random_probabilities(self.num_hidden)).float() # ∈ R^(64 × 128)
        
        # Входные данные транспонируются и умножаются на вектор из 0 и 1, полученный выше
        # input_data^T ∈ R^(784 × 64), positive_hidden_activations ∈ R^(64 × 128)
        positive_associations = torch.matmul(input_data.t(), positive_hidden_activations) # ∈ R^(784 × 128)

        # Задаем начальный вектор h для алгоритма контрастной дивергенции
        hidden_activations = positive_hidden_activations  # ∈ R^(64 × 128)
        
        # Цикл по шагам алгоритма контрастной дивергенции
        for step in range(self.k):
            # Вычисляем вектор p(v|h)
            visible_probabilities = self.sample_visible(hidden_activations) # ∈ R^(64 × 784)
            
            # Вычисляем вектор p(h|v), на основе предыдущего вектора
            hidden_probabilities = self.sample_hidden(visible_probabilities) # ∈ R^(64 × 128)
            
            # Активируем скрытые нейроны и обновляем вектор h
            hidden_activations = (hidden_probabilities >= self._random_probabilities(self.num_hidden)).float() # ∈ R^(64 × 128)
        
        # Там, где стоит probabilities, вектора содержат вероятности от 0 до 1, а где activations - только 0 или 1
        
        # Запоминаем в отдельные переменные последние полученные вектора вероятностей p(v|h) и p(h|v)
        negative_visible_probabilities = visible_probabilities # ∈ R^(64 × 784)
        negative_hidden_probabilities = hidden_probabilities # ∈ R^(64 × 128)
        
        # Аналогично вычислению positive_associations
        negative_associations = torch.matmul(negative_visible_probabilities.t(), negative_hidden_probabilities)# ∈ R^(784 × 128)

        # Обновление параметров
        # Пересчет вспомогательной матрицы весов
        self.weights_momentum *= self.momentum_coefficient # ∈ R^(784 × 128)
        self.weights_momentum += (positive_associations - negative_associations) # ∈ R^(784 × 128)
        
        # Пересчет вспомогательного вектора сдвига видимых нейронов
        self.visible_bias_momentum *= self.momentum_coefficient # ∈ R^(1 × 784)
        self.visible_bias_momentum += torch.sum(input_data - negative_visible_probabilities, dim=0) # ∈ R^(1 × 784)
        
        # Пересчет вспомогательного вектора сдвига скрытых нейронов
        self.hidden_bias_momentum *= self.momentum_coefficient # ∈ R^(1 × 128)
        self.hidden_bias_momentum += torch.sum(positive_hidden_probabilities - negative_hidden_probabilities, dim=0)
        # ∈ R^(1 × 128)
        
        # Запоминаем текущий размер обрабатываемой пачки данных
        batch_size = input_data.size(0)
        
        # Обновление основных параметров модели (матрицы весов и векторов сдвигов вдимых и скрытых нейронов)
        self.weights += self.weights_momentum * self.learning_rate / batch_size
        self.visible_bias += self.visible_bias_momentum * self.learning_rate / batch_size
        self.hidden_bias += self.hidden_bias_momentum * self.learning_rate / batch_size
        
        # Решение проблемы затухания весов
        self.weights -= self.weights * self.weight_decay  

        # Вычисление квадратичной ошибки
        error = torch.sum((input_data - negative_visible_probabilities)**2)

        return error
    

    # Задаем сигмоидальную функцию: 1 / (1 + e^(-x))
    def _sigmoid(self, x):
        return 1 / (1 + torch.exp(-x))
    
    # Данная функция создает вектор длины num, заполняет его случайными 
    # числами из стандартного нормального распределения и возвращает его
    def _random_probabilities(self, num): 
        return torch.rand(num)
    
    # Одно нижнее подчеркивание фактически означает, что данные методы 
    # имеют спецификатор доступа protected (но это считается на уровне 
    # соглашения, данные методы можно вызвать вне класса)

__Создадим объект разработанного класса:__

In [14]:
rbm = RBM(visible_units, hidden_units, CD_k, learning_rate, momentum_coefficient, weight_decay)

### 3. Обучение построенной модели

__Обучим модель:__

In [15]:
%%time
# Цикл по эпохам
for epoch in range(num_epochs):
    # Полученная ошибка для текущей эпохи
    epoch_error = 0.0
    
    # Проход по всем пачкам данных
    for batch, _ in train_loader: 
        # Формирование пачки данных
        batch = batch.view(len(batch), visible_units)
        # Вычисление ошибки алгоритма контрастной дивергенции
        # на текущей пачке данных
        batch_error = rbm.contrastive_divergence(batch)
        # Обновление ошибки текущей эпохи
        epoch_error += batch_error

    print('Epoch error (epoch = %d): %.4f' % (epoch + 1, epoch_error))

Epoch error (epoch = 1): 8969122.0000
Epoch error (epoch = 2): 5041285.0000
Epoch error (epoch = 3): 4109471.5000
Epoch error (epoch = 4): 3563144.0000
Epoch error (epoch = 5): 3180193.7500
Epoch error (epoch = 6): 2892092.5000
Epoch error (epoch = 7): 2663455.5000
Epoch error (epoch = 8): 2476906.0000
Epoch error (epoch = 9): 2323418.7500
Epoch error (epoch = 10): 2197613.2500
Wall time: 1min 26s


In [16]:
# X_train, y_train
train_features = np.zeros((len(train_dataset), hidden_units))
train_labels = np.zeros(len(train_dataset))

# X_test, y_test
test_features = np.zeros((len(test_dataset), hidden_units))
test_labels = np.zeros(len(test_dataset))

# Цикл по обучающей выборке
for i, (batch, labels) in enumerate(train_loader):
    # Формирование пачки данных
    batch = batch.view(len(batch), visible_units)
    # В X_train записываются p(h|v), полученные с помощью обученной RBM
    train_features[i*batch_size:i*batch_size+len(batch)] = rbm.sample_hidden(batch).numpy()
    # В y_train присваиваются метки классов
    train_labels[i*batch_size:i*batch_size+len(batch)] = labels.numpy()

# Цикл по тестовой выборке
for i, (batch, labels) in enumerate(test_loader):
    # Формирование пачки данных
    batch = batch.view(len(batch), visible_units)
    # В X_test записываются p(h|v), полученные с помощью обученной RBM
    test_features[i*batch_size:i*batch_size+len(batch)] = rbm.sample_hidden(batch).numpy()
    # В y_test присваиваются метки классов
    test_labels[i*batch_size:i*batch_size+len(batch)] = labels.numpy()

In [17]:
# Логистическая регрессия
model = LogisticRegression(solver = 'lbfgs', max_iter = 1000).fit(train_features, train_labels)
predictions = model.predict(test_features)

err_train = np.mean(train_labels != model.predict(train_features))
err_test  = np.mean(test_labels  != model.predict(test_features))

print('Ошибка на обучающей выборке: ', err_train)
print('Ошибка на тестовой выборке: ', err_test)

Ошибка на обучающей выборке:  0.0897
Ошибка на тестовой выборке:  0.0938
